In [ ]:
!pip install -q openai-whisper nltk scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 43.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import whisper
import nltk
from nltk.tokenize import sent_tokenize
from google.colab import files

In [ ]:
print("Upload your MP3 file")
uploaded = files.upload()

Upload your MP3 file


Saving a6.mp3 to a6.mp3


In [ ]:
audio_path = list(uploaded.keys())[0]
print("Uploaded file:", audio_path)

Uploaded file: a6.mp3


In [ ]:
print("Loading Whisper model...")
model = whisper.load_model("base")

Loading Whisper model...


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 331MiB/s]


In [ ]:
print("Transcribing audio... This may take a few minutes.")
result = model.transcribe(audio_path)
transcript = result["text"]

print("\nTranscription completed! First 500 characters:\n")
print(transcript[:50000])

Transcribing audio... This may take a few minutes.

Transcription completed! First 500 characters:

 Hello, hello, hello, and welcome to English Learning for Curious Mines by Leonardo English. The show where you can listen to fascinating stories and learn weird and wonderful things about the world at the same time as improving your English. I'm Alistair Budge, and today it's the start of our next three-part mini-series this time on the theme of university. In part one, today's episode will go on a whirlwind journey through the history of universities and higher education, from the medieval period to the present day. In part two, we'll zoom in and talk about universities in the UK, the different types of universities, what life is like at a British university, and I'll share a load of my personal experience too, as I spent a total of five years studying at universities in the UK. And in part three, the final part will talk about something that said it was a university, but was later acc

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
sentences = sent_tokenize(transcript)
print("\nTotal sentences:", len(sentences))


Total sentences: 151


In [ ]:
paragraphs = []
sentences_per_paragraph = 4
for i in range(0, len(sentences), sentences_per_paragraph):
    para = " ".join(sentences[i:i + sentences_per_paragraph])
    paragraphs.append(para)

print("Total paragraphs created:", len(paragraphs))

Total paragraphs created: 38


In [ ]:
output_file = audio_path.replace(".mp3", "_paragraphs.txt")
with open(output_file, "w", encoding="utf-8") as f:
    for para in paragraphs:
        f.write(para + "\n\n")

print(f"\nParagraph-formatted transcript saved as {output_file}")


Paragraph-formatted transcript saved as a6_paragraphs.txt


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
vectorizer = TfidfVectorizer(stop_words="english")
tfidf = vectorizer.fit_transform(paragraphs)  # paragraphs from previous step


In [ ]:
similarities = []

for i in range(len(paragraphs)-1):
    sim = cosine_similarity(tfidf[i], tfidf[i+1])[0][0]
    similarities.append(sim)

# Display first 10 similarities for reference
print("Cosine similarities between consecutive paragraphs:", similarities[:10])


Cosine similarities between consecutive paragraphs: [np.float64(0.023969575237950402), np.float64(0.0899703757863382), np.float64(0.19734422769528923), np.float64(0.011760743367069896), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.015584176772610097), np.float64(0.063722251382557), np.float64(0.06350727272924173)]


In [ ]:
threshold = 0.45  # lower similarity = likely topic change
topic_boundaries = [i+1 for i, sim in enumerate(similarities) if sim < threshold]

print("Topic boundaries at paragraph numbers:", topic_boundaries)


Topic boundaries at paragraph numbers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]


In [ ]:
topics = []
start = 0

for boundary in topic_boundaries:
    topics.append(" ".join(paragraphs[start:boundary]))
    start = boundary

# Add last segment
topics.append(" ".join(paragraphs[start:]))

print("Total topics detected:", len(topics))


Total topics detected: 38


In [ ]:
output_file = audio_path.replace(".mp3", "_topics.txt")
with open(output_file, "w", encoding="utf-8") as f:
    for i, topic in enumerate(topics, 1):
        f.write(f"--- Topic {i} ---\n")
        f.write(topic + "\n\n")

print(f"Topic segmentation saved as {output_file}")


Topic segmentation saved as a6_topics.txt


In [ ]:
from google.colab import files
files.download("a6_topics.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Open the topic segmentation file
with open("a6_topics.txt", "r", encoding="utf-8") as f:
    topics_content = f.read()

# Split by topics (assuming each topic starts with "--- Topic")
topics = topics_content.split('--- Topic ')[1:]  # skip any empty text before first topic

# Print each topic fully
for t in topics:
    lines = t.split('\n', 1)  # first line = topic number
    topic_num = lines[0].strip()
    topic_text = lines[1].strip() if len(lines) > 1 else ""

    print(f"\n🔹 Topic {topic_num}\n{topic_text}\n")



🔹 Topic 1 ---
Hello, hello, hello, and welcome to English Learning for Curious Mines by Leonardo English. The show where you can listen to fascinating stories and learn weird and wonderful things about the world at the same time as improving your English. I'm Alistair Budge, and today it's the start of our next three-part mini-series this time on the theme of university. In part one, today's episode will go on a whirlwind journey through the history of universities and higher education, from the medieval period to the present day.


🔹 Topic 2 ---
In part two, we'll zoom in and talk about universities in the UK, the different types of universities, what life is like at a British university, and I'll share a load of my personal experience too, as I spent a total of five years studying at universities in the UK. And in part three, the final part will talk about something that said it was a university, but was later accused of being a complete fraud Trump university. Oh, okay then, let's 